In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
import numpy as np
import matplotlib.pyplot as plt

# Function to load and preprocess content and style images
def load_and_preprocess_image(path, target_size=(224, 224)):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img)
    img = tf.image.resize(img, target_size)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return tf.expand_dims(img, axis=0)

# 1. Load sample data (use Naruto content and style images)
content_image_path = '/content/wp4642409-naruto-manga-wallpapers.jpg'  # Replace with your custom content image path
style_image_path = '/content/ancient-japan-background-digital-art-style.jpg.crdownload'    # Replace with your custom style image path

content_image = load_and_preprocess_image(content_image_path)
style_image = load_and_preprocess_image(style_image_path)

# 2. VGG19 model for extracting features (same as before)
vgg = VGG19(include_top=False, weights='imagenet')

def get_vgg_model(vgg, content_layer, style_layers):
    content_output = vgg.get_layer(content_layer).output
    style_outputs = [vgg.get_layer(layer).output for layer in style_layers]
    return tf.keras.Model(inputs=vgg.input, outputs=[content_output] + style_outputs)

content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']

vgg_model = get_vgg_model(vgg, content_layer, style_layers)
content_output = vgg_model(content_image)
style_output = vgg_model(style_image)

# Generate an initial image to optimize
generated_image = tf.Variable(content_image, dtype=tf.float32)

# Compute loss and train step (same as before)
def compute_loss(generated_image, content_output, style_output):
    generated_features = vgg_model(generated_image)
    content_loss = tf.reduce_mean((generated_features[0] - content_output[0]) ** 2)
    style_loss = 0
    for i in range(len(style_layers)):
        generated_style = generated_features[i + 1]
        style = style_output[i + 1]
        gram_generated = tf.linalg.einsum('bijc,bijd->bcd', generated_style, generated_style)
        gram_style = tf.linalg.einsum('bijc,bijd->bcd', style, style)
        gram_generated /= tf.cast(tf.shape(generated_style)[1] * tf.shape(generated_style)[2], tf.float32)
        gram_style /= tf.cast(tf.shape(style)[1] * tf.shape(style)[2], tf.float32)
        style_loss += tf.reduce_mean((gram_generated - gram_style) ** 2)
    total_loss = content_loss + (0.01 * style_loss)
    return total_loss

optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)

@tf.function()
def train_step(generated_image, content_output, style_output):
    with tf.GradientTape() as tape:
        loss = compute_loss(generated_image, content_output, style_output)
    gradients = tape.gradient(loss, generated_image)
    optimizer.apply_gradients([(gradients, generated_image)])
    generated_image.assign(tf.clip_by_value(generated_image, 0.0, 255.0))

# Train the model
epochs = 1000
for epoch in range(epochs):
    train_step(generated_image, content_output, style_output)
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {compute_loss(generated_image, content_output, style_output)}")
        img = generated_image.numpy()
        img = np.squeeze(img, axis=0)
        img = img.astype('uint8')
        plt.imshow(img)
        plt.show()


InvalidArgumentError: {{function_node __wrapped__DecodeImage_device_/job:localhost/replica:0/task:0/device:CPU:0}} Invalid PNG data, size 1048576 [Op:DecodeImage] name: 